In [1]:
#測試從 mlFLow 把訓練好的模型拉回來跑 classify
import sklearn
from sklearn import datasets
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn import preprocessing
from sklearn.preprocessing import OneHotEncoder
from sklearn.neural_network import MLPClassifier

import mlflow
import mlflow.sklearn
from mlflow.tracking import MlflowClient

In [2]:
#讀 data 和訓練時一樣
iris = datasets.load_iris()
X = iris.data
y = iris.target
print(X[0:5])
print(y[0:5])
#X做scale
X = preprocessing.scale(X)
print(X[0:5])
#y轉one hot
oneHotEncoder = preprocessing.OneHotEncoder()
y = oneHotEncoder.fit_transform(y.reshape(-1, 1)).toarray() #OneHotEncoder吃2D，然後y是1D，所以要reshape
print(y[0:5])
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2)

[[5.1 3.5 1.4 0.2]
 [4.9 3.  1.4 0.2]
 [4.7 3.2 1.3 0.2]
 [4.6 3.1 1.5 0.2]
 [5.  3.6 1.4 0.2]]
[0 0 0 0 0]
[[-0.90068117  1.01900435 -1.34022653 -1.3154443 ]
 [-1.14301691 -0.13197948 -1.34022653 -1.3154443 ]
 [-1.38535265  0.32841405 -1.39706395 -1.3154443 ]
 [-1.50652052  0.09821729 -1.2833891  -1.3154443 ]
 [-1.02184904  1.24920112 -1.34022653 -1.3154443 ]]
[[1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]]


In [3]:
#這邊不訓練直接拉模型回來
#mlp = mlflow.sklearn.load_model('file:///home/jovyan/work/mlruns/1/52d6eab4e4564bf7903e77ed22535dfe/artifacts/model')
#更改為從註冊的模型抓 list而非寫死 full path
mlflow.set_tracking_uri("http://localhost:5000")
client = MlflowClient()

max_version = -1.0
for msg in client.search_model_versions("name='iris_classify_test'"):
    #print(msg.source)
    #print(msg.version)
    if max_version < float(msg.version):
        max_version = float(msg.version)
        targetSource = msg.source
        modelInfo = msg
if max_version > 0:
    print('load model: \n' + str(modelInfo))
    mlp = mlflow.sklearn.load_model(modelInfo.source)

load model: 
<ModelVersion: creation_timestamp=1615399481272, current_stage='None', description='', last_updated_timestamp=1615399481272, name='iris_classify_test', run_id='3ed5896d641f473885463c76649486c1', run_link='', source='/home/jovyan/work/artifacts/1/3ed5896d641f473885463c76649486c1/artifacts/model', status='READY', status_message='', tags={}, user_id='', version='2'>


In [4]:
#預測
predict_train = mlp.predict(X_train)
predict_test = mlp.predict(X_test)
train_acc = sklearn.metrics.accuracy_score(predict_train, y_train)
test_acc = sklearn.metrics.accuracy_score(predict_test, y_test)
print(mlp)
print("train accuracy: %f" % (train_acc))
print("test accuracy: %f\n\n" % (test_acc))

MLPClassifier(alpha=0.001, hidden_layer_sizes=(10, 10, 10), max_iter=1000,
              random_state=1234, solver='lbfgs')
train accuracy: 0.958333
test accuracy: 0.966667


